In [1]:
import mysql.connector
import pandas as pd

In [4]:
query = ("SELECT DISTINCT MRREL.AUI2, MRREL.AUI1, MRREL.REL, SAB FROM MRREL WHERE (REL = 'PAR' or REL='CHD') and (SAB LIKE '%MSH%' OR SAB LIKE '%OMIM%')")

conn = mysql.connector.connect(user='umls', password='umls', database='umls2019ab', host='138.26.120.27')
cursor = conn.cursor()

In [5]:
cursor.execute(query)

In [6]:
total_relations_umls = []
for x in cursor:
    total_relations_umls.append(x)

In [7]:
df = pd.DataFrame(total_relations_umls, columns =['FirstAUI', 'SecondAUI', 'REL','SAB'])

In [8]:
query2 = ("SELECT AUI, SAB, CODE, STR FROM MRCONSO WHERE (SAB LIKE '%MSH%' OR SAB LIKE '%OMIM%') and ISPREF = 'Y'")

In [9]:
cursor.execute(query2)

In [10]:
names_of_CUIS = []
for x in cursor:
    names_of_CUIS.append(x)

In [11]:
df2 = pd.DataFrame(names_of_CUIS, columns =['AUI', 'SAB', 'CODE','STR'])

In [12]:
result = df.merge(df2, left_on="FirstAUI", right_on="AUI")

In [13]:
result = result.drop(['SAB_x','SAB_y', 'AUI'], axis=1)

In [14]:
result = result.merge(df2, left_on="SecondAUI", right_on="AUI")

In [15]:
result = result.drop(['AUI','SAB', 'AUI'], axis=1)

In [16]:
result.columns = ['FirstAUI', 'SecondAUI', 'REL', 'CODE_First', 'STR_First', 'CODE_Second','STR_Second']

In [17]:
result = result.groupby(['CODE_First','REL', 'STR_First'])['CODE_Second'].apply(list).reset_index()

In [18]:
result

,CODE_First,REL,STR_First,CODE_Second
0,D000001,CHD,Calcimycin,[D001583]
1,D000002,CHD,Temefos,[D063086]
2,D000003,CHD,Abattoirs,"[D000072499, D008462]"
3,D000004,CHD,Abbreviations as Topic,[D009275]
4,D000007,PAR,Abdominal Injuries,"[D013161, D006549, D013275]"
...,...,...,...,...
50926,U000020,PAR,Publication Type,[U000023]
50927,U000021,CHD,Check Tag,[U000017]
50928,U000021,PAR,Check Tag,"[D005260, D008297]"
50929,U000023,CHD,Publication Characteristics,[U000020]


In [19]:
result.to_csv('UMLSgraph.csv')

In [20]:
with open('MSHOMIMVocab.txt', 'w') as f:
    for i,v in df2.iterrows():
        f.write(f'{v["CODE"]}||{v["STR"]}\n')